## Downloading Hadoop

In [1]:
%env HADOOP_VERSION hadoop-2.9.2

env: HADOOP_VERSION=hadoop-2.9.2


In [2]:
!wget http://ftp.unicamp.br/pub/apache/hadoop/common/${HADOOP_VERSION}/${HADOOP_VERSION}.tar.gz -q --show-progress

hadoop-2.9.2.tar.gz 100%[===================>] 349.47M  15.0MB/s    in 37s     


### Extracting compressed files and removing .tar

In [3]:
!tar -xvf ${HADOOP_VERSION}.tar.gz >/dev/null 
!rm       ${HADOOP_VERSION}.tar.gz

### Discovering Java path

In [4]:
!dirname $(dirname $(readlink -f $(which javac)))

/usr/lib/jvm/java-8-openjdk-amd64


### Setting Java path envvar

We also added it to user's .bashrc so it will be loaded as the nodes perform ssh connections.

In [5]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64

env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [6]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " > .bashrc

# Using Hadoop in Standalone Mode (local)

# Using Hadoop in Pseudo-Distributed Mode

### Starting sshd server

Check postBuild and sshd_config files for more details

In [7]:
!/usr/sbin/sshd -f resources/configs/ssh/sshd_config 

### Adding to know hosts by establishing a ssh connectcion (avoiding yes/no host confirmation)

In [8]:
!ssh -o "StrictHostKeyChecking no" $USER@localhost -p 8822 -C "exit" 
!ssh -o "StrictHostKeyChecking no" $USER@0.0.0.0   -p 8822 -C "exit"

### Copying configurations files to Hadoop folder

In [9]:
!cp resources/configs/hadoop/* ${HADOOP_VERSION}/etc/hadoop/

### Formating the filesystem

In [ ]:
!./${HADOOP_VERSION}/bin/hdfs namenode -format -force -nonInteractive

### Adding ssh options: running in a diferent port

In [11]:
%env HADOOP_SSH_OPTS -p 8822 

env: HADOOP_SSH_OPTS=-p 8822


### Starting/stoping NameNode daemon and DataNode daemon

In [12]:
!./${HADOOP_VERSION}/sbin/start-dfs.sh
#!./${HADOOP_VERSION}/sbin/stop-dfs.sh

Starting namenodes on [localhost]
localhost: starting namenode, logging to /home/jovyan/hadoop-2.9.2/logs/hadoop-jovyan-namenode-jupyter-thedatasociety-2dmybinder-2dhadoop-2drwzpz5yc.out
localhost: starting datanode, logging to /home/jovyan/hadoop-2.9.2/logs/hadoop-jovyan-datanode-jupyter-thedatasociety-2dmybinder-2dhadoop-2drwzpz5yc.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /home/jovyan/hadoop-2.9.2/logs/hadoop-jovyan-secondarynamenode-jupyter-thedatasociety-2dmybinder-2dhadoop-2drwzpz5yc.out


### Creating folders in the distributed file system

In [13]:
!./${HADOOP_VERSION}/bin/hdfs dfs -mkdir /user/
!./${HADOOP_VERSION}/bin/hdfs dfs -mkdir /user/matheus/
!./${HADOOP_VERSION}/bin/hdfs dfs -mkdir /user/matheus/input/

### Copying a file to a folder in the distributed file system

In [14]:
!./${HADOOP_VERSION}/bin/hdfs dfs -put ./resources/examples/newyorknewyork.txt /user/matheus/input/

In [15]:
!./${HADOOP_VERSION}/bin/hdfs dfs -ls /user/matheus/input/

Found 1 items
-rw-r--r--   1 jovyan supergroup        865 2019-06-24 03:42 /user/matheus/input/newyorknewyork.txt


In [16]:
!./${HADOOP_VERSION}/bin/hdfs dfs -cat /user/matheus/input/newyorknewyork.txt

Start spreading the news
I am leaving today
I want to be a part of it
New York New York
These vagabond shoes
They are longing to stray
Right through the very heart of it
New York New York
I want to wake up in that city
That doesn't sleep
And find I'm king of the hill
Top of the heap
My little town blues
They are melting away
I gonna make a brand new start of it
In old New York
If I can make it there
I'll make it anywhere
It's up to you
New York New York
New York New York
I want to wake up in that city
That never sleeps
And find I'm king of the hill
Top of the list
Head of the heap
King of the hill
These are little town blues
They have all melted away
I am about to make a brand new start of it
Right there in old New York
And you bet baby
If I can make it there
You know I'm gonna make it just about anywhere
Come on come through
New York New York New York


# Running YARN on a single node

In [17]:
!./${HADOOP_VERSION}/sbin/start-yarn.sh

starting yarn daemons
starting resourcemanager, logging to /home/jovyan/hadoop-2.9.2/logs/yarn-jovyan-resourcemanager-jupyter-thedatasociety-2dmybinder-2dhadoop-2drwzpz5yc.out
localhost: starting nodemanager, logging to /home/jovyan/hadoop-2.9.2/logs/yarn-jovyan-nodemanager-jupyter-thedatasociety-2dmybinder-2dhadoop-2drwzpz5yc.out


# Simple word count example

In [18]:
!./${HADOOP_VERSION}/bin/hadoop jar  ./${HADOOP_VERSION}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.9.2.jar wordcount \
                                      /user/matheus/input /user/matheus/output

19/06/24 03:43:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/24 03:43:32 INFO input.FileInputFormat: Total input files to process : 1
19/06/24 03:43:33 INFO mapreduce.JobSubmitter: number of splits:1
19/06/24 03:43:34 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
19/06/24 03:43:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1561347791672_0001
19/06/24 03:43:37 INFO impl.YarnClientImpl: Submitted application application_1561347791672_0001
19/06/24 03:43:38 INFO mapreduce.Job: The url to track the job: http://jupyter-thedatasociety-2dmybinder-2dhadoop-2drwzpz5yc:8088/proxy/application_1561347791672_0001/
19/06/24 03:43:38 INFO mapreduce.Job: Running job: job_1561347791672_0001
19/06/24 03:43:56 INFO mapreduce.Job: Job job_1561347791672_0001 running in uber mode : false
19/06/24 03:43:56 INFO mapreduce.Job:  map 0% reduce 0%
19/06/24 03:4

In [19]:
!./${HADOOP_VERSION}/bin/hdfs dfs -ls /user/matheus/output/

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2019-06-24 03:44 /user/matheus/output/_SUCCESS
-rw-r--r--   1 jovyan supergroup        571 2019-06-24 03:44 /user/matheus/output/part-r-00000


In [20]:
!./${HADOOP_VERSION}/bin/hdfs dfs -cat /user/matheus/output/part-r-00000

And	3
Come	1
Head	1
I	8
I'll	1
I'm	3
If	2
In	1
It's	1
King	1
My	1
New	13
Right	2
Start	1
That	2
These	2
They	3
Top	2
York	13
You	1
a	3
about	2
all	1
am	2
anywhere	2
are	3
away	2
baby	1
be	1
bet	1
blues	2
brand	2
can	2
city	2
come	1
doesn't	1
find	2
gonna	2
have	1
heap	2
heart	1
hill	3
in	3
it	8
just	1
king	2
know	1
leaving	1
list	1
little	2
longing	1
make	6
melted	1
melting	1
never	1
new	2
news	1
of	10
old	2
on	1
part	1
shoes	1
sleep	1
sleeps	1
spreading	1
start	2
stray	1
that	2
the	8
there	3
through	2
to	6
today	1
town	2
up	3
vagabond	1
very	1
wake	2
want	3
you	2


In [21]:
# !./${HADOOP_VERSION}/bin/hdfs dfs -rm -r  /user/matheus/output/